In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import requests
from IPython.display import display , HTML

In [3]:
products=pd.read_csv('main data/amz_ca_total_products_data_processed.csv')

In [4]:
products.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,categoryName,isBestSeller,boughtInLastMonth
0,B07CV4L6HX,"Green Leaf WW3D Wonder Extension Cord Winder, ...",https://m.media-amazon.com/images/I/81cRe0AVC4...,https://www.amazon.ca/dp/B07CV4L6HX,4.4,2876,47.69,0.0,Industrial Scientific,False,0
1,B09N1HGY74,8pcs Toilet Seat Bumpers Universal Toilet Repl...,https://m.media-amazon.com/images/I/512CpB-EP5...,https://www.amazon.ca/dp/B09N1HGY74,3.8,55,10.99,0.0,Industrial Scientific,False,100


In [16]:
products=pd.read_csv('main data\mobile_recommendation_system_dataset.csv')
products["index"]=products.index = range(1, len(products) + 1)

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\tesla\AppData\Local\Temp\ipykernel_19088\3340268451.py:1: SyntaxWarning: invalid escape sequence '\m'
  products=pd.read_csv('main data\mobile_recommendation_system_dataset.csv')


In [17]:
products.head(2)

,name,ratings,price,imgURL,corpus,index
1,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,1
2,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,2
